In [2]:
import os
import random
import yaml
import subprocess
import platform
from collections import defaultdict

# Define dataset directory
DIR_PATH = "slakh2100_flac_redux/reduced_test"

# Load metadata.yaml from a track folder
def load_metadata(track_path):
    yaml_path = os.path.join(track_path, "metadata.yaml")
    if not os.path.exists(yaml_path):
        return None
    with open(yaml_path, "r") as f:
        return yaml.safe_load(f)

In [6]:
def summarize_plugins_by_class(inst_class):
    plugin_counts = defaultdict(int)
    total_stems = 0

    track_folders = [f for f in os.listdir(DIR_PATH) if f.startswith("Track")]
    if not track_folders:
        print("No tracks found.")
        return

    for track in track_folders:
        track_path = os.path.join(DIR_PATH, track)
        metadata_path = os.path.join(track_path, "metadata.yaml")

        if not os.path.isfile(metadata_path):
            continue

        try:
            with open(metadata_path, "r") as file:
                metadata = yaml.safe_load(file)
                stems = metadata.get("stems", {})

                for stem_data in stems.values():
                    if stem_data.get("inst_class") == inst_class:
                        plugin_name = stem_data.get("plugin_name", "Unknown")
                        plugin_counts[plugin_name] += 1
                        total_stems += 1

        except yaml.YAMLError as e:
            print(f"Error parsing {metadata_path}: {e}")

    print(f"Total Stems: {total_stems}\n")
    for plugin, count in sorted(plugin_counts.items(), key=lambda x: x[0].lower()):
        percentage = (count / total_stems) * 100 if total_stems > 0 else 0
        print(f"{plugin}: {count} ({percentage:.2f}%)")

# Example usage
summarize_plugins_by_class("Guitar")

Total Stems: 389

/Library/Audio/Plug-Ins/Components/AGML2.component: 46 (11.83%)
elektrik_guitar.nkm: 24 (6.17%)
funk_guitar.nkm: 33 (8.48%)
harmonic_guitar.nkm: 20 (5.14%)
jazz_guitar.nkm: 27 (6.94%)
jazz_guitar2.nkm: 21 (5.40%)
jazz_guitar3.nkm: 56 (14.40%)
jazz_guitar4.nkm: 23 (5.91%)
nylon_guitar.nkm: 43 (11.05%)
nylon_guitar2.nkm: 42 (10.80%)
rhythm_rock_guitar.nkm: 17 (4.37%)
rock_guitar.nkm: 15 (3.86%)
solo_guitar.nkm: 22 (5.66%)


In [21]:
# Search stems by program_num or plugin_name
def find_matching_stems(metadata, program_num=None, plugin_name=None):
    stems = metadata.get("stems", {}) if metadata else {}
    return [
        (stem_id, stem)
        for stem_id, stem in stems.items()
        if (program_num is not None and stem.get("program_num") == program_num) or
           (plugin_name is not None and stem.get("plugin_name") == plugin_name)
    ]

# Open a folder in file explorer
def open_file_explorer(path):
    match platform.system():
        case "Windows": os.startfile(path)
        case "Darwin": subprocess.run(["open", path])
        case "Linux": subprocess.run(["xdg-open", path])
        case _: print(f"Unsupported OS: {platform.system()}")

# Find a random track that matches a given program_num or plugin_name
def find_random_track(program_num=None, plugin_name=None):
    if not (program_num or plugin_name):
        print("Specify either program_num or plugin_name.")
        return

    track_folders = [f for f in os.listdir(DIR_PATH) if f.startswith("Track")]
    if not track_folders:
        print("No tracks found.")
        return

    random.shuffle(track_folders)
    for track in track_folders:
        track_path = os.path.join(DIR_PATH, track)
        matches = find_matching_stems(load_metadata(track_path), program_num, plugin_name)

        if matches:
            print(f"Track: {track}")
            for stem_id, stem in matches:
                print(f"  Stem: {stem_id}")
                print(f"    program_num: {stem.get('program_num', 'N/A')}")
                print(f"    plugin_name: {stem.get('plugin_name', 'N/A')}")
            open_file_explorer(track_path)
            return

    print("No matching stems found.")

# find_random_track(program_num=6)
find_random_track(plugin_name="jazz_guitar3.nkm")

Track: Track01940
  Stem: S03
    program_num: 28
    plugin_name: jazz_guitar3.nkm
  Stem: S05
    program_num: 26
    plugin_name: jazz_guitar3.nkm
